In [27]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage import filters
import scipy.ndimage as sci_image
from skimage import measure as meas
import matplotlib.pyplot as plt
from scipy import signal as sci_signal
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import os
import warnings
warnings.filterwarnings("ignore")
import supported_functions as sf

In [29]:
#char_list = list(range(48,58)) + list(range(65,91))
#ctr = 0
#for i in char_list:
    #file_name = "data/" + chr(i) + ".bmp"
    #print(ctr," : ",sf.detect_letters(cv2.imread(file_name)))
    #ctr += 1

In [34]:
image = cv2.imread("test/image1.png")
image_resized = cv2.resize(image, (24,42))
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image_binarize = image_grayscale.copy()

image_binarize[image_binarize >= 128] = 255
image_binarize[image_binarize < 128] = 0
image_binarize

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [36]:
image_resized.shape

(42, 24, 3)